In [ ]:
import pandas as pd 
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
import keras

import numpy as np

def build_nn(input_dim, hidden_dim, output_dim, layer_num, activation, dropout_rate):
    inputs = keras.Input(shape = (input_dim, ))

    x = keras.layers.Dense(hidden_dim, activation = activation)(inputs)
    for i in range(layer_num):
        x = keras.layers.Dense(hidden_dim, activation = activation)(x)
        x = keras.layers.Dropout(dropout_rate)(x)
    
    outputs = keras.layers.Dense(output_dim)(x)

    return outputs

In [ ]:
class RankNetGenerator(object):
    def __init__(self, X, y, race_index_list, X_scaler= None, batch_size = 32):
        self._X = X
        self._y = y
        self._race_index_list = race_index_list
        self._X_scaler = X_scaler
        self._batch_size = batch_size

    def __iter__(self):
        return self
    
    def __len__(self):
        return len(self._X)
    
    def __next__(self):
        race_indexes = np.random.randint(0, len(self._race_index_list), self._batch_size) # 0~len(race_index_list)の乱数をbatch_size回生成
        race_indexes = [self._race_index_list[race_index].values for race_index in race_indexes] 

        horse_indexes = [race_index[np.random.choice(race_index.shape[0], 2, replace=False)] for race_index in race_indexes] # np.random.choiceはrace_index.shape[0]から2つランダムにとってくる、重複不可
        horse_indexes = [(idx_1, idx_2) if self._y.loc[idx_1].values < self._y.loc[idx_2].values else (idx_2, idx_1) for idx_1, idx_2 in horse_indexes]

        X = np.array([(self._X.loc[horse_index[0]], self._X.loc[horse_index[1]]) for horse_index in horse_indexes])
        X = [self._X_scaler.transform(X[:, 0, :]), self._X_scaler.transform(X[:, 1, :])]

        y = np.ones(self._batch_size)

        return X, y    

In [ ]:
def build_ranknet(input_dim, hidden_dim, output_dim, layer_num, activation, dropout_rate):
    inputs_1 = keras.Input(shape=(input_dim,))
    inputs_2 = keras.Input(shape=(input_dim,))

    nn = build_nn(input_dim=input_dim,
                  hidden_dim=hidden_dim,
                  output_dim=output_dim,
                  layer_num=layer_num,
                  activation=activation,
                  dropout_rate=dropout_rate)
    
    x1 = nn(inputs_1)
    x2 = nn(inputs_2)

    subtract = keras.layers.Subtract()([x1, x2])
    outputs = keras.layers.Activation('sigmoid')(subtract)

    return keras.Model(inputs=[inputs_1, inputs_2], outputs=outputs)

In [ ]:
from wsgiref.validate import validator


input_dim = 284
output_dim = 1
hidden_dim = 128
layer_num = 3
activation = tf.nn.relu
dropout_rate = 0.1
epoch = 100
train_steps = 1000
valid_step = 100
loss = "binary_crossentropy"
optimizer = "adam"

X_scaler = StandardScaler()
_ = X_scaler.fit_transform(train_X.values)

train_generator = RankNetGenerator(train_X, train_y, train_index_list, X_scaler=X_scaler, batch_size=batch_size)
valid_generator = RankNetGenerator(valid_X, valid_y, valid_index_list, X_scaler=X_scaler, batch_size=batch_size)

ranknet = build_ranknet(input_dim, hidden_dim, output_dim, layer_num, activation, dropout_rate)

ranknet.compile(optimizer=optimizer, loss=loss)

callbacks = [keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)]

self._history = ranknet.fit(train_generator,
                            validation_data=valid_generator,
                            epochs=epoch,
                            batch_size=batch_size,
                            callbacks=callbacks,
                            steps_per_epoch=train_steps,
                            validation_steps=valid_step)


2024-12-02 16:41:38.774075: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-02 16:41:38.805769: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-12-02 16:41:39.078162: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-12-02 16:41:39.315621: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1733125299.496109   13227 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1733125299.54